In [77]:
import openfermion
from src.q_systems import ElectronicSystem
from electronic_system_hamiltonians import  ham_14_qubits, ham_16_qubits
import numpy
from src.vqe_runner import VQERunner
from src.ansatz_elements import *
from src.backends import *
from src.utils import LogUtils
from src.cache import *
from pylab import *
import pandas
from src.backends import QiskitSimBackend
from qiskit import Aer
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister 
from qiskit.quantum_info import Pauli
from qiskit.aqua.operators.legacy import pauli_measurement
from qiskit import IBMQ
from qiskit.compiler import transpile


In [12]:
n_orbitals = 14
n_electrons = 10

In [13]:
operator = openfermion.FermionOperator('[0^ 0 2^ 2]-[0^ 0 3^ 3]-[1^ 1 2^ 2]+[1^ 1 3^ 3]')
operator = openfermion.jordan_wigner(operator)
operator

(0.25+0j) [Z0 Z2] +
(-0.25+0j) [Z0 Z3] +
(-0.25+0j) [Z1 Z2] +
(0.25+0j) [Z1 Z3]

In [14]:
def long_stupid_ansatz():

    ansatz = []
    # ansatz.append(Gen2QubitAnsatzElement(0, 1))

    for i in range(n_orbitals):
        for j in range(i + 1, n_orbitals):
            if i % 2 == j % 2:
                ansatz.append(EffSFExc(i, j, system_n_qubits=n_orbitals))

    for i, j, k, l in itertools.combinations(range(n_orbitals), 4):
        # spin conserving excitations only
        if i % 2 + j % 2 == k % 2 + l % 2:
            ansatz.append(DFExc([i, j], [k, l], system_n_qubits=n_orbitals))

    return ansatz

In [15]:
ansatz =long_stupid_ansatz()
parameters = [ 7.70153256e-17,  2.10180158e-17,  1.76063693e-16,  1.15775938e-16,
        2.65884412e-01,  4.56728308e-01,  2.40061494e-16, -2.39070899e-18,
        1.29689944e-16,  1.14963921e-16,  1.95019293e-01,  4.18619125e-01,
        1.73299760e-17, -9.12865839e-17,  2.88831075e-17, -5.30627915e-01,
       -2.52021838e-01, -4.26379394e-17,  4.73992789e-17, -1.58525459e-17,
       -5.38922216e-01, -2.83032548e-01, -1.55296877e-17, -7.75782373e-18,
        1.89005773e-01, -1.09781102e-01,  4.80988268e-17,  1.52359549e-17,
        1.78197395e-01, -1.23610762e-01, -1.29605262e-17, -4.55647749e-01,
       -4.83579598e-01, -2.07871736e-17, -4.32582476e-01, -4.99081669e-01,
       -1.17647827e-01,  6.23731641e-02, -1.07247420e-01,  8.59775377e-02,
       -1.85845156e-02, -7.09683485e-02, -2.39079042e-01,  5.53030187e-03,
        4.60450091e-02,  9.92244760e-02, -7.85427764e-02, -9.31384916e-04,
        5.57077174e-03, -5.91023611e-02, -9.36741291e-02,  1.07363961e-01,
        1.34630332e-02, -1.33189253e-03,  1.49392822e-02,  5.30376691e-03,
       -2.63698222e-02, -3.27383144e-02, -9.45674101e-03, -4.67275891e-03,
        2.47032751e-02,  3.44450506e-02,  2.31133320e-02,  2.20672804e-02,
       -3.89302324e-02, -6.47970641e-02, -3.00628177e-02,  8.56844138e-02,
        7.78327424e-02,  4.06416859e-03, -2.23962220e-02, -4.83774290e-02,
        2.35652123e-02,  4.01403206e-02,  5.19803685e-02,  9.46250763e-02,
       -6.30287969e-02,  1.07036215e-01,  2.18103124e-03,  1.49311877e-03,
        3.73478468e-02, -1.63585933e-03,  1.00195682e-02, -1.02473237e-02,
       -1.51576570e-02, -7.42417225e-04,  1.58853977e-03, -3.14975086e-02,
        7.62979989e-03, -4.33799840e-02, -1.79966000e-02,  6.36265191e-02,
        3.89162314e-02,  4.25868443e-02,  1.74511558e-02, -2.54616999e-02,
       -4.26217736e-02,  2.05717192e-01,  3.92933299e-02, -1.34340883e-01,
       -1.27979385e-01, -2.43759641e-02,  9.05266716e-02,  1.00967853e-01,
        2.34791761e-02, -3.57066432e-02, -2.18058705e-03,  3.89141983e-02,
        5.33570265e-03,  1.32766857e-01,  7.76498098e-02, -1.32717857e-01,
        8.16956431e-02, -6.68412305e-03, -1.19987008e-03,  5.53574770e-03,
       -3.10004200e-03,  4.71638665e-03, -6.31369538e-04, -8.94802965e-03,
       -1.09620061e-02,  6.11985934e-03,  1.12625419e-03,  3.94294952e-03,
        6.80041130e-03,  2.73335365e-03, -7.12071716e-03,  5.18294298e-04,
        6.12209712e-03, -3.06940615e-03, -2.80667186e-03,  2.45173602e-03,
       -4.70554225e-03,  1.83430049e-02, -4.97599626e-03, -1.45111696e-02,
        1.70457478e-02, -3.76858438e-02,  5.57264618e-03, -2.46122435e-03,
        2.77718707e-02, -9.96890909e-03, -1.05992210e-02, -5.32205941e-02,
       -5.99789330e-03, -2.59533093e-02,  8.92418842e-02, -1.43769288e-03,
       -1.08735714e-01, -7.00892297e-02,  5.85237287e-02, -6.26550663e-02,
        6.94389038e-02, -3.83205517e-02,  1.39673942e-01, -4.09858458e-02,
       -9.06686085e-02, -1.57488185e-01, -1.08366615e-02,  5.30691234e-02,
        1.96533621e-02, -4.99927274e-02, -5.20856648e-02, -4.55432160e-02,
       -1.83067539e-02,  6.84900117e-02,  3.23244878e-02, -2.11672976e-01,
       -4.08538199e-02,  1.36073946e-01,  1.15037642e-01,  3.95923492e-02,
       -1.48601450e-01, -1.04887425e-01, -2.30326534e-02,  4.93145018e-02,
        1.00058751e-02, -4.49274649e-02,  7.41233222e-03, -1.65738766e-01,
       -1.31403244e-01,  7.50812497e-02, -6.34521506e-02,  5.47362955e-03,
        2.12040178e-03,  3.57868905e-02, -3.37220203e-03,  1.02118963e-02,
       -3.40204495e-03, -1.69652211e-02, -4.70356618e-03,  9.71900892e-03,
       -3.68698035e-02,  3.86709125e-03,  4.96705146e-03, -1.98137943e-02,
        4.81169126e-03,  1.63700029e-02,  9.47335336e-03, -6.27182663e-03,
       -2.10208076e-03, -1.37529424e-02,  2.08215029e-02,  1.86327635e-02,
        6.90875199e-03, -1.41652119e-02, -1.00545806e-02,  5.52439019e-03,
        3.07283706e-03,  2.63893931e-02,  2.68301596e-02, -9.27128221e-05,
        2.52404614e-03, -7.30906252e-03, -9.20645510e-05,  1.43629685e-02,
       -4.79902408e-03, -2.59987346e-03,  2.68868231e-02,  1.05429573e-02,
       -2.01956719e-02, -9.05884555e-02, -9.27172639e-03, -7.21362917e-03,
       -7.47319518e-03, -5.72682593e-03,  1.78419076e-02,  4.32753148e-02,
       -5.20031166e-02,  4.69903381e-02, -9.65353708e-02, -1.35399880e-02,
        1.47012216e-02, -1.85967695e-02,  9.16988779e-03,  5.41251155e-03,
        6.11077185e-02,  2.75892967e-02, -5.51800400e-02,  3.46132338e-02,
       -2.83090238e-02,  7.39770905e-02, -5.89089365e-02,  1.44806758e-02,
       -2.91365886e-02,  6.72672591e-03,  4.56168785e-02, -2.52266784e-02,
        3.63540137e-02, -6.31438208e-03, -5.52568380e-02,  3.69885665e-03,
        8.95198477e-02,  2.65085925e-02, -2.42098335e-01, -2.32290441e-02,
       -2.97768296e-02,  2.01782590e-01,  3.45737330e-02, -1.62350403e-02,
       -2.70280716e-02, -1.50282763e-02,  2.58252513e-02,  3.32586901e-02,
        2.66891150e-01,  1.78048328e-02, -3.01623433e-02,  2.13906645e-02,
        1.04156637e-02, -1.16113936e-02, -5.94134367e-03, -1.60776801e-02,
        2.43692618e-02, -3.62309947e-02, -3.94728240e-03,  8.62491593e-03,
        1.59124884e-02,  1.37713716e-02,  1.81099235e-03, -1.17481431e-02,
       -2.58605268e-02,  1.77245587e-02, -3.77761481e-02,  2.33118332e-02,
        2.12277836e-02, -5.92177421e-03,  4.92602361e-02, -5.05153210e-03,
       -2.46244613e-02, -2.15667665e-02, -2.17264073e-02,  4.90561302e-02,
       -8.13801147e-02, -1.85539094e-02,  2.30218198e-02,  1.53784743e-02,
       -4.23217317e-02,  4.94784385e-02, -8.02430481e-02, -3.85315400e-02,
       -4.78295895e-02,  9.60595189e-02,  2.78062792e-02, -1.52544660e-01,
       -1.70365894e-03,  3.36108368e-02,  3.36932096e-02, -8.39365746e-02,
       -4.53696210e-03, -7.64213488e-03,  1.01867370e-02,  9.87407264e-04,
        8.19987061e-04, -5.97879369e-03, -1.38967165e-02, -1.80887397e-02,
        8.21592272e-03, -7.82658411e-03,  1.11886250e-02, -1.59326092e-02,
        2.58874242e-02,  3.29310536e-05, -3.39442780e-02,  2.41632978e-02,
       -6.03939034e-02, -2.12427187e-02,  1.83945574e-03,  1.55035570e-02,
        3.98759715e-03,  7.31399365e-05, -6.88450516e-03, -1.34120720e-02,
        1.34290001e-02, -2.55200390e-02, -1.50127970e-03,  3.17473301e-03,
       -6.39441841e-02,  2.44104488e-02, -4.39943367e-02,  4.41438646e-02,
        2.77261853e-02, -1.41177817e-03,  7.12986867e-02, -5.06528955e-02,
       -4.67483144e-02, -8.17680047e-02, -2.42760968e-02,  4.35605576e-02,
       -6.89621569e-02,  3.85095891e-02, -7.20208903e-03,  1.56141049e-01,
        2.27176784e-02, -1.23567682e-02, -3.40262678e-02,  1.79809320e-02,
       -1.17896135e-01,  9.30329959e-02,  1.30639243e-03, -2.31311712e-03,
       -7.03068229e-03, -3.69583253e-03,  9.65526988e-04,  3.41230112e-03,
        4.31595238e-04, -3.19259232e-03,  5.12130373e-03,  2.43488773e-03,
       -3.75063690e-03, -1.81980435e-03, -1.27054687e-02, -6.78968649e-03,
        1.17937852e-02,  6.20919986e-03,  1.20709894e-02, -2.12296340e-02,
        5.26542525e-02,  9.38316671e-03, -1.49207026e-02, -5.05313927e-03,
        2.34279306e-02, -2.21629567e-02,  3.63710468e-02,  1.67594053e-02,
        3.31539140e-02, -4.34360562e-02, -1.77951425e-02, -1.43294098e-02,
       -1.02265893e-03, -8.86996004e-03,  8.57331509e-03, -3.80800827e-02,
       -1.05570284e-03,  1.18625234e-02,  4.07870686e-02, -1.10644625e-02,
        2.31168985e-02, -1.58849371e-02, -1.06838927e-02, -3.26514360e-03,
       -2.83795369e-02,  2.40550571e-02,  2.56496076e-02,  4.68575353e-02,
        1.38512791e-02, -1.93762187e-02,  4.58388597e-02, -9.02232592e-04,
        3.88282539e-03,  1.75661815e-02, -1.15407659e-02, -1.63328813e-03,
       -1.02238500e-02, -1.19703604e-02, -4.45966880e-03,  3.57427857e-02,
       -1.13169541e-02,  8.74981631e-03, -3.73001164e-02, -1.04187266e-02,
        3.94977113e-02,  1.21262900e-02,  3.49418108e-02, -4.93499308e-02,
        5.27311584e-02, -6.60899096e-03, -2.84248142e-03, -3.53836045e-04,
        2.20050527e-03, -4.63261085e-03, -7.92844340e-02,  8.67386923e-03,
       -8.03994235e-03,  1.91391579e-02, -5.40781951e-03, -3.05243223e-03,
        7.14580723e-03, -1.82123746e-03, -9.79237537e-02,  6.61378792e-02,
        4.73865612e-02, -9.77856844e-03,  2.62029871e-03, -1.03449855e-02,
        2.88476851e-02,  8.79569612e-03, -3.22909393e-02, -9.38486083e-02]

In [17]:
# qasm for the trial state
ansatz_qasm = ['']
ansatz_qasm.append(QasmUtils.qasm_header(n_orbitals))
ansatz_qasm.append(QasmUtils.hf_state(n_electrons))
ansatz_qasm += QiskitSimBackend.qasm_from_ansatz(ansatz, parameters)
ansatz_qasm = ''.join(ansatz_qasm)

# measurement circuits for each of the the four Pauli terms of the operator we want to measure
measurements_qasms = ['cx q[0], q[2];\n', 'cx q[0], q[3];\n', 'cx q[1], q[2];\n', 'cx q[1], q[3];\n']

In [75]:
def meas_probs_of_single_qubit(qasm, qubit=0):
    circ = QuantumCircuit.from_qasm_str(qasm)
    circ.measure([qubit], [qubit])
    
#     backend_sim = Aer.get_backend('qasm_simulator')
    IBMQ.save_account('0c694b7d1006259f5149bc9f572b070d12542355f02ddb58999e1b774d14a41fef0c9b40bc1c974beb750c5d46e251575cb991c66262b0d680d94ce9e4c22f9f')
    IBMQ.load_account()

    backend_name = 'ibmq_guadalupe'
    provider = IBMQ.get_provider()
    backend_sim= provider.get_backend(backend_name)
    
    shots =1024
    job_sim = backend_sim.run(transpile(circ, backend_sim), shots=shots)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(circ)
    
    # magic
    key_0 = '0'*n_orbitals
    key_1 = list(key_0)
    key_1[n_orbitals - qubit - 1] = '1'
    key_1 = ''.join(key_1)
    
    return counts[key_0]/ shots, counts[key_1] / shots
    
# dummy_qasm = 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[14];\ncreg c[14];\n rx(1.57) q[0];' 
# meas_probs_of_single_qubit(dummy_qasm, 0)


In [78]:
# measure the operator expectation value

exp_value = 0

operator_coefficients = [0.25, -0.25, - 0.25, 0.25]
measured_qubits = [2,3,2,3]

for i in range(4):
    print(i)
    probs = meas_probs_of_single_qubit(ansatz_qasm + measurements_qasms[i], measured_qubits[i])
    exp_value += operator_coefficients[i]*(probs[0] - probs[1])

exp_value



0


configrc.store_credentials:WARNING:2021-12-26 15:20:23,607: Credentials already present. Set overwrite=True to overwrite.


QiskitBackendNotFoundError: 'No backend matches the criteria'